In [ ]:
import os  # 운영체제와 상호작용하기 위한 모듈
import numpy as np  # 수치 계산을 위한 모듈
import rasterio  # 래스터 이미지를 처리하기 위한 모듈
import matplotlib.pyplot as plt  # 이미지 시각화를 위한 모듈
from skimage.transform import resize  # 이미지를 리사이즈하기 위한 모듈
from tqdm import tqdm  # 진행 상황을 시각적으로 보여주기 위한 모듈

# 입력 디렉토리와 출력 디렉토리 경로 설정
input_directory = 'your_path'  # 원본 이미지가 저장된 디렉토리
output_directory = 'your_path'  # 변환된 이미지가 저장될 디렉토리

# 출력 디렉토리가 없으면 생성
def create_directory(directory_path):
    """디렉토리가 존재하지 않으면 생성"""
    if not os.path.exists(directory_path):  # 출력 디렉토리가 존재하지 않으면
        os.makedirs(directory_path)  # 출력 디렉토리 생성

# 주어진 폴더에서 필요한 밴드 파일을 찾는 함수
def find_band_files(folder_path, required_bands):
    """폴더에서 필요한 밴드 파일 경로를 반환"""
    band_files = {band: None for band in required_bands}  # 각 밴드에 대한 파일 경로를 저장할 딕셔너리
    for f in os.listdir(folder_path):  # 폴더 내의 모든 파일을 순회
        if f.endswith('.tif'):  # 파일이 .tif로 끝나는지 확인
            for band in required_bands:  # 필요한 밴드 목록 순회
                if band in f:  # 파일 이름에 밴드 이름이 포함되어 있는지 확인
                    band_files[band] = os.path.join(folder_path, f)  # 해당 밴드의 파일 경로 저장
    return band_files  # 찾은 밴드 파일 경로 반환

# 밴드 파일을 읽고 정규화하는 함수
def read_and_normalize_band(file_path):
    """밴드 파일을 읽고 0-1 범위로 정규화"""
    with rasterio.open(file_path) as dataset:  # 래스터 파일 열기
        band_data = dataset.read(1).astype(float)  # 첫 번째 밴드 읽기 및 float 타입으로 변환
        return (band_data - np.min(band_data)) / (np.max(band_data) - np.min(band_data))  # 정규화 후 반환

# RGB 이미지를 생성하고 저장하는 함수
def create_and_save_rgb_image(bands_data, output_file_path):
    """밴드 데이터를 기반으로 RGB 이미지 생성 후 저장"""
    rgb_image = np.dstack((bands_data['B04'], bands_data['B03'], bands_data['B02']))  # B04, B03, B02 밴드를 쌓아 RGB 이미지 생성
    rgb_image_resized = resize(rgb_image, (256, 256), anti_aliasing=True)  # 이미지 크기 조정
    plt.imsave(output_file_path, rgb_image_resized)  # 이미지 저장

# 폴더를 처리하여 RGB 이미지로 변환하는 함수
def process_folder(folder_path, output_folder, required_bands):
    """폴더를 처리하여 RGB 이미지로 변환하고 저장"""
    band_files = find_band_files(folder_path, required_bands)  # 필요한 밴드 파일 찾기
    
    if all(band_files.values()):  # 모든 밴드 파일이 존재하는지 확인
        bands_data = {band: read_and_normalize_band(file_path) for band, file_path in band_files.items()}  # 밴드 데이터 읽고 정규화
        
        output_file_path = os.path.join(output_folder, os.path.basename(folder_path) + '_rgb.png')  # 출력 파일 경로 설정
        create_and_save_rgb_image(bands_data, output_file_path)  # RGB 이미지 생성 및 저장

# 모든 폴더를 순회하여 처리하는 함수
def process_all_folders(input_directory, output_directory, required_bands):
    """입력 디렉토리 내 모든 폴더를 처리하여 RGB 이미지로 변환"""
    create_directory(output_directory)  # 출력 디렉토리 생성

    for first_level_folder in tqdm(os.listdir(input_directory), desc="Processing first level folders"):  # 첫 번째 레벨 폴더 목록을 순회
        first_level_path = os.path.join(input_directory, first_level_folder)  # 첫 번째 레벨 폴더의 전체 경로 설정
        if os.path.isdir(first_level_path):  # 경로가 디렉토리인지 확인
            for second_level_folder in tqdm(os.listdir(first_level_path), desc="Processing second level folders", leave=False):  # 두 번째 레벨 폴더 목록을 순회
                second_level_path = os.path.join(first_level_path, second_level_folder)  # 두 번째 레벨 폴더의 전체 경로 설정
                if os.path.isdir(second_level_path):  # 경로가 디렉토리인지 확인
                    process_folder(second_level_path, output_directory, required_bands)  # 두 번째 레벨 폴더를 처리하여 RGB 이미지로 변환하고 출력 디렉토리에 저장

# 메인 함수 호출
process_all_folders(input_directory, output_directory, ['B02', 'B03', 'B04'])  # 모든 폴더를 처리하여 RGB 이미지로 변환